# Data Collection

In [1]:
import time
import requests
import os
import serpapi
import pandas as pd
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv

## Importing Fear and Greed index data

In [ ]:
def fetch_fear_and_greed_index(limit=1, format='json', date_format='us'):
    # Base URL for the Fear and Greed Index API
    base_url = "https://api.alternative.me/fng/"

    url = base_url + f"?limit={limit}&format={format}&date_format={date_format}"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Extracting historical data
        historical_data = data['data']

        # Converting data into pandas DataFrame
        df = pd.DataFrame(historical_data, columns=['value', 'value_classification', 'timestamp'])
        
        # Converting timestamp to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m-%d-%Y')
        
        # timestamp as index
        df.set_index('timestamp', inplace=True)

        return df
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

if __name__ == "__main__":
    limit = 0  
    format = 'json'  
    date_format = 'us'  
    
    fear_and_greed_index_data = fetch_fear_and_greed_index(limit=limit, format=format, date_format=date_format)
    
    if fear_and_greed_index_data is not None:
        fear_and_greed_index_data.to_csv('fear_and_greed_index_data.csv')


## Importing Google Trends Data 

In [72]:
load_dotenv()
api_key = os.getenv('SERPAPI_KEY')
client = serpapi.Client(api_key=api_key)

def search_google_trends(start_date, end_date):
    params = {
        'engine': 'google_trends',
        'q': 'bitcoin',
        'api_key': api_key,
        'geo': '',  
        'date': f'{start_date} {end_date}',
        'tz': '420',
        'data_type': 'TIMESERIES',
        'interval': 'daily'  
    }
    return client.search(**params)

start_year = 2016
end_year = 2024
eight_months = timedelta(days=8*30)


dates = []
timestamps = []
values = []

# Iterate over each 8-month period
current_date = datetime(start_year, 1, 1)
while current_date.year < end_year:
    start_date = current_date.strftime('%Y-%m-%d')
    end_date = (current_date + eight_months).strftime('%Y-%m-%d')
    
    search_result = search_google_trends(start_date, end_date)
    timeline_data = search_result['interest_over_time']['timeline_data']
    
    # Extracting data from the search result
    for entry in timeline_data:
        date = entry['date']
        timestamp = entry['timestamp']
        for value_entry in entry['values']:
            value = value_entry['value']
            # Append data to lists
            dates.append(date)
            timestamps.append(timestamp)
            values.append(value)
    
    # next 8-month period
    current_date += eight_months


data_dict = {
    'date': dates,
    'timestamp': timestamps,
    'value': values
}

df = pd.DataFrame(data_dict)
df['timestamp'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')

print(df)


              date  timestamp value
0      Jan 1, 2016 2016-01-01    29
1      Jan 2, 2016 2016-01-02    31
2      Jan 3, 2016 2016-01-03    33
3      Jan 4, 2016 2016-01-04    33
4      Jan 5, 2016 2016-01-05    36
...            ...        ...   ...
3017  Mar 24, 2024 2024-03-24    40
3018  Mar 25, 2024 2024-03-25    52
3019  Mar 26, 2024 2024-03-26    49
3020  Mar 27, 2024 2024-03-27    46
3021  Mar 28, 2024 2024-03-28    43

[3022 rows x 3 columns]


In [73]:
df.to_csv('bitcoin_gt.csv')

## Filtering data from 2021-2022

In [29]:
fng = pd.read_csv('fear_and_greed_index_data.csv')

In [31]:
# Filtering the DataFrame from January 1, 2021, and June 30, 2022
filtered_data = fng[(fng['timestamp'] <= '2022-06-30') & (fng['timestamp'] >= '2021-01-01')]

filtered_data.reset_index(drop=True, inplace=True)

filtered_data = filtered_data.sort_values(by='timestamp').reset_index(drop=True)

In [30]:
filtered_data.to_csv('FnG_index_21_22.csv')

In [2]:
filtered_data = pd.read_csv('FnG_index_21_22.csv', index_col=0)

In [3]:
filtered_data

,timestamp,value,value_classification
0,2021-01-01,94,Extreme Greed
1,2021-01-02,94,Extreme Greed
2,2021-01-03,93,Extreme Greed
3,2021-01-04,94,Extreme Greed
4,2021-01-05,93,Extreme Greed
...,...,...,...
541,2022-06-26,14,Extreme Fear
542,2022-06-27,12,Extreme Fear
543,2022-06-28,10,Extreme Fear
544,2022-06-29,13,Extreme Fear


In [ ]:
btc_gt = pd.read_csv('bitcoin_gt.csv', index_col=0)

In [46]:
btc_gt

,date,timestamp,value
0,"Jan 1, 2016",2016-01-01,29
1,"Jan 2, 2016",2016-01-02,31
2,"Jan 3, 2016",2016-01-03,33
3,"Jan 4, 2016",2016-01-04,33
4,"Jan 5, 2016",2016-01-05,36
...,...,...,...
3017,"Mar 24, 2024",2024-03-24,40
3018,"Mar 25, 2024",2024-03-25,52
3019,"Mar 26, 2024",2024-03-26,49
3020,"Mar 27, 2024",2024-03-27,46


In [33]:
gt_filtered = btc_gt[(btc_gt['timestamp'] <= '2022-06-30') & (btc_gt['timestamp'] >= '2021-01-01')]

gt_filtered.reset_index(drop=True, inplace=True)

gt_filtered = gt_filtered.sort_values(by='timestamp').reset_index(drop=True)

In [48]:
gt_filtered

,date,timestamp,value
0,"Jan 1, 2021",2021-01-01,39
1,"Jan 2, 2021",2021-01-02,64
2,"Jan 3, 2021",2021-01-03,75
3,"Jan 4, 2021",2021-01-04,64
4,"Jan 5, 2021",2021-01-05,54
...,...,...,...
543,"Jun 26, 2022",2022-06-26,38
544,"Jun 27, 2022",2022-06-27,43
545,"Jun 28, 2022",2022-06-28,42
546,"Jun 29, 2022",2022-06-29,44


In [ ]:
gt_filtered.to_csv('btc_gt_21_22.csv')

In [7]:
gt_filtered = pd.read_csv('btc_gt_21_22.csv', index_col=0)

In [8]:
gt_filtered

,date,timestamp,value
0,"Jan 1, 2021",2021-01-01,39
1,"Jan 2, 2021",2021-01-02,64
2,"Jan 3, 2021",2021-01-03,75
3,"Jan 4, 2021",2021-01-04,64
4,"Jan 5, 2021",2021-01-05,54
...,...,...,...
543,"Jun 26, 2022",2022-06-26,38
544,"Jun 27, 2022",2022-06-27,43
545,"Jun 28, 2022",2022-06-28,42
546,"Jun 29, 2022",2022-06-29,44


## Importing BTC Hourly Historic Data

In [22]:
btc = pd.read_csv('BTCUSDT_H1.csv')
btc['datetime'] = pd.to_datetime(btc['datetime'])


In [23]:
# Calculate 24h% Change
btc['24h_change'] = btc['close'].pct_change(periods=24) * 100
btc['24h_change'] = btc['24h_change'].apply(lambda x: f"{x:.2f}" if pd.isnull(x) or x >= 0 else f"{x:.2f}")

# Calculate 7d% Change
btc['7d_change'] = btc['close'].pct_change(periods=24*7) * 100
btc['7d_change'] = btc['7d_change'].apply(lambda x: f"{x:.2f}" if pd.isnull(x) or x >= 0 else f"{x:.2f}")

# Calculate 30d% Change
btc['30d_change'] = btc['close'].pct_change(periods=24*30) * 100
btc['30d_change'] = btc['30d_change'].apply(lambda x: f"{x:.2f}" if pd.isnull(x) or x >= 0 else f"{x:.2f}")

In [24]:
btc['next_day_pct_change'] = btc['close'].pct_change(periods=-24) * 100
btc['next_7days_pct_change'] = btc['close'].pct_change(periods=-24*7) * 100
btc['next_30days_pct_change'] = btc['close'].pct_change(periods=-24*30) * 100

In [25]:
def classify_movement(change, threshold_up, threshold_down):
    if pd.isnull(change):
        return 0
    if change >= threshold_up:
        return 1
    elif change <= threshold_down:
        return -1
    else:
        return 0

btc['next_1d_movement'] = btc['next_day_pct_change'].apply(lambda x: classify_movement(x, 2, -2))
btc['next_7d_movement'] = btc['next_7days_pct_change'].apply(lambda x: classify_movement(x, 5, -5))
btc['next_30d_movement'] = btc['next_30days_pct_change'].apply(lambda x: classify_movement(x, 10, -10))

btc.drop(columns=['next_day_pct_change', 'next_7days_pct_change', 'next_30days_pct_change'], inplace=True)

In [26]:
btc

,datetime,open,high,low,close,volume,24h_change,7d_change,30d_change,next_1d_movement,next_7d_movement,next_30d_movement
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,nan,nan,nan,0,0,1
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,nan,nan,nan,0,0,1
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,nan,nan,nan,0,0,1
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,nan,nan,nan,0,0,1
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,nan,nan,nan,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
57506,2024-03-14 14:00:00,71884.82,72536.03,71215.00,71690.00,4628.078380,-0.92,6.96,47.23,0,0,0
57507,2024-03-14 15:00:00,71689.99,72068.96,70543.64,70844.01,5356.465000,-2.47,5.33,45.39,0,0,0
57508,2024-03-14 16:00:00,70844.00,71445.00,69880.00,71337.63,6369.771090,-2.27,5.99,47.16,0,0,0
57509,2024-03-14 17:00:00,71337.64,71458.00,70700.00,71036.01,2805.732360,-2.45,5.09,45.43,0,0,0


In [27]:
btc_filter = btc[(btc['datetime'] < '2022-07-01') & (btc['datetime'] >= '2021-01-01')]
btc_filter.reset_index(drop=True, inplace=True)
btc_filter = btc_filter.sort_values(by='datetime').reset_index(drop=True)

In [28]:
btc_filter['date'] = pd.to_datetime(btc_filter['datetime']).dt.date
filtered_data['timestamp'] = pd.to_datetime(filtered_data['timestamp']).dt.date
gt_filtered['timestamp'] = pd.to_datetime(gt_filtered['timestamp']).dt.date

In [30]:
btc_filter.rename(columns={'date': 'timestamp'}, inplace=True)
merged_df = pd.merge(btc_filter, filtered_data, how='inner', on='timestamp')
merged_df = pd.merge(merged_df, gt_filtered, how='inner', on='timestamp')

In [31]:
merged_df.rename(columns={'value_x': 'fng_value',
                         'value_classification': 'fng_classification',
                         'value_y': 'btc_gt'}, inplace=True)

In [32]:
merged_df.drop(columns=['date'], inplace=True)

In [33]:
merged_df.to_csv('hourly_combined_data.csv')

In [34]:
merged_df

,datetime,open,high,low,close,volume,24h_change,7d_change,30d_change,next_1d_movement,next_7d_movement,next_30d_movement,timestamp,fng_value,fng_classification,btc_gt
0,2021-01-01 00:00:00,28923.63,29031.34,28690.17,28995.13,2311.811445,-0.43,22.19,52.05,0,-1,-1,2021-01-01,94,Extreme Greed,39
1,2021-01-01 01:00:00,28995.13,29470.00,28960.35,29409.99,5403.068471,2.32,24.50,54.59,0,-1,-1,2021-01-01,94,Extreme Greed,39
2,2021-01-01 02:00:00,29410.00,29465.26,29120.03,29194.65,2384.231560,1.65,23.72,53.35,0,-1,-1,2021-01-01,94,Extreme Greed,39
3,2021-01-01 03:00:00,29195.25,29367.00,29150.02,29278.40,1461.345077,0.88,24.69,54.95,0,-1,-1,2021-01-01,94,Extreme Greed,39
4,2021-01-01 04:00:00,29278.41,29395.00,29029.40,29220.31,2038.046803,0.66,23.75,55.72,0,-1,-1,2021-01-01,94,Extreme Greed,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13134,2022-06-30 19:00:00,18973.86,19046.39,18850.21,18927.34,3385.782190,-6.41,-9.48,-40.15,-1,-1,-1,2022-06-30,11,Extreme Fear,52
13135,2022-06-30 20:00:00,18927.33,19075.49,18626.00,18755.93,4502.282250,-7.26,-9.89,-41.04,-1,-1,-1,2022-06-30,11,Extreme Fear,52
13136,2022-06-30 21:00:00,18755.93,18967.57,18724.18,18819.80,2580.288140,-7.27,-9.03,-40.67,-1,-1,-1,2022-06-30,11,Extreme Fear,52
13137,2022-06-30 22:00:00,18819.79,18957.08,18740.83,18894.54,3006.627190,-6.53,-9.99,-40.71,-1,-1,-1,2022-06-30,11,Extreme Fear,52
